In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
import hopsworks
import pandas as pd
from datetime import timedelta
from src.inference import get_feature_store, fetch_predictions

def fetch_hourly_rides(hours):
    current_hour = (pd.Timestamp.now(tz="Etc/UTC") - timedelta(hours=hours)).floor('h')

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_NAME,
        version=1
    )

    query = fg.select_all()
    query = query.filter(fg.pickup_hour >= current_hour)

    return query.read()

In [4]:
df = fetch_hourly_rides(12)

2025-03-03 23:22:26,247 INFO: Initializing external client
2025-03-03 23:22:26,249 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 23:22:28,119 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214680
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.46s) 


In [5]:
df

,pickup_hour,pickup_location_id,rides
0,2025-03-03 12:00:00+00:00,193,4
1,2025-03-03 12:00:00+00:00,2,0
2,2025-03-03 15:00:00+00:00,134,0
3,2025-03-03 12:00:00+00:00,199,0
4,2025-03-03 14:00:00+00:00,61,4
...,...,...,...
4262,2025-03-03 21:00:00+00:00,237,269
4263,2025-03-03 21:00:00+00:00,122,0
4264,2025-03-03 21:00:00+00:00,231,56
4265,2025-03-03 21:00:00+00:00,40,0


In [6]:
df_pred = fetch_predictions(12)

2025-03-03 23:23:34,641 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-03 23:23:34,652 INFO: Initializing external client
2025-03-03 23:23:34,652 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 23:23:35,500 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214680
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.46s) 


In [7]:
df_pred

,pickup_location_id,predicted_demand,pickup_hour
0,28,0.0,2025-03-04 01:00:00+00:00
1,128,0.0,2025-03-04 01:00:00+00:00
2,7,2.0,2025-03-04 01:00:00+00:00
3,130,1.0,2025-03-04 01:00:00+00:00
4,19,0.0,2025-03-04 01:00:00+00:00
...,...,...,...
497,117,0.0,2025-03-04 05:00:00+00:00
498,73,0.0,2025-03-04 05:00:00+00:00
499,174,0.0,2025-03-04 05:00:00+00:00
500,258,0.0,2025-03-04 05:00:00+00:00


In [10]:
merged_df = pd.merge(df, df_pred, on=['pickup_location_id', 'pickup_hour'])

In [11]:
merged_df

,pickup_hour,pickup_location_id,rides,predicted_demand
0,2025-03-04 01:00:00+00:00,132,76,40.0
1,2025-03-04 01:00:00+00:00,41,0,15.0
2,2025-03-04 01:00:00+00:00,138,8,41.0
3,2025-03-04 01:00:00+00:00,210,0,0.0
4,2025-03-04 01:00:00+00:00,38,0,0.0
...,...,...,...,...
246,2025-03-04 01:00:00+00:00,225,0,1.0
247,2025-03-04 01:00:00+00:00,113,6,30.0
248,2025-03-04 01:00:00+00:00,68,10,121.0
249,2025-03-04 01:00:00+00:00,222,0,0.0


In [12]:
merged_df.sort_values(["pickup_location_id", "pickup_hour"])


,pickup_hour,pickup_location_id,rides,predicted_demand
94,2025-03-04 01:00:00+00:00,2,0,0.0
63,2025-03-04 01:00:00+00:00,3,0,0.0
139,2025-03-04 01:00:00+00:00,4,4,2.0
58,2025-03-04 01:00:00+00:00,6,0,0.0
141,2025-03-04 01:00:00+00:00,7,1,2.0
...,...,...,...,...
42,2025-03-04 01:00:00+00:00,259,0,0.0
131,2025-03-04 01:00:00+00:00,260,1,0.0
52,2025-03-04 01:00:00+00:00,261,0,16.0
112,2025-03-04 01:00:00+00:00,262,2,96.0


In [13]:
import pandas as pd  
import plotly.express as px
df1 = df
df2 = df_pred

# Merge the DataFrames on 'pickup_location_id' and 'pickup_hour'  
merged_df = pd.merge(df1, df2, on=['pickup_location_id', 'pickup_hour'])  

# Calculate the absolute error  
merged_df['absolute_error'] = abs(merged_df['predicted_demand'] - merged_df['rides'])  

# Group by 'pickup_hour' and calculate the mean absolute error (MAE)  
mae_by_hour = merged_df.groupby('pickup_hour')['absolute_error'].mean().reset_index()  
mae_by_hour.rename(columns={'absolute_error': 'MAE'}, inplace=True)  

# Create a Plotly plot  
fig = px.line(  
    mae_by_hour,  
    x='pickup_hour',  
    y='MAE',  
    title='Mean Absolute Error (MAE) by Pickup Hour',  
    labels={'pickup_hour': 'Pickup Hour', 'MAE': 'Mean Absolute Error'},  
    markers=True  
)  

# Show the plot  
fig.show()